In [6]:
import pandas as pd
import numpy as np
# Step 1: Load the Data
df = pd.read_parquet('E:/680_temp/Daily_temp/temp_merge_data.parquet')

In [7]:
df['year'].unique() # check the unique values of year

array([1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010], dtype=int64)

In [12]:
# define the thresholds and country list
thresholds = [99, 95, 90, 85, 80, 75]
cntrylist = [188, 214, 222, 332, 388, 558, 591]

# calculate the quantile of each threshold
for threshold in thresholds:
    df[f'threshold_{threshold}'] = df.groupby(['country', 'dist'])['temp_min'].transform(lambda x: x.quantile(threshold/100.0))

# whether exceed the threshold
for threshold in thresholds:
    df[f'exceed_{threshold}'] = (df['temp_min'] > df[f'threshold_{threshold}']).astype(int)

# calculate the number of days exceed the threshold
for threshold in thresholds:
    df[f'nday_exceed_{threshold}'] = df.groupby(['country', 'dist', 'year'])[f'exceed_{threshold}'].transform('sum')

# calculate rolling mean and std
for threshold in thresholds:
    df[f'rolling_mean_{threshold}'] = df.groupby(['country', 'dist'])[f'nday_exceed_{threshold}'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())
    df[f'rolling_std_{threshold}'] = df.groupby(['country', 'dist'])[f'nday_exceed_{threshold}'].transform(lambda x: x.rolling(window=5, min_periods=1).std())

# calculate z-score
for threshold in thresholds:
    df[f'z_{threshold}'] = (df[f'nday_exceed_{threshold}'] - df[f'rolling_mean_{threshold}']) / df[f'rolling_std_{threshold}']

# choose the columns to save
columns_to_save = ['country', 'dist', 'year'] + [f'z_{threshold}' for threshold in thresholds]
df_final = df[columns_to_save]

output_path = "E:/680_temp/Daily_temp/z_withmex.csv"  # replace with your save path
df.to_csv(output_path, index=False)
#df.to_parquet('E:/680_temp/Daily_temp/z_withmex.parquet')